# Imports

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib
#import urllib3.request,sys,time
from urllib.error import HTTPError
from urllib.request import urlopen
import pandas as pd
import time
import warnings 
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
warnings.filterwarnings('ignore')

/home/queant/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
driver = webdriver.Chrome()
driver.get("https://www.telegraph.co.uk/")

# Test de Selenium avec Facebook 

In [335]:
driver.get("https://www.telegraph.co.uk/search/")

In [336]:
search_bar = driver.find_element("name", "fsearch")
search_bar.send_keys("facebook")
search_bar.send_keys(Keys.ENTER)

In [337]:
pagemax = int(driver.find_element("class name", "cst-pagination__label").text[5:])

On a la fonction générale, en partant de la page de recherche : 

In [322]:
def getLinks(theme : str) :
    driver.get("https://www.telegraph.co.uk/search/")
    search_bar = driver.find_element("name", "fsearch")
    search_bar.send_keys(theme)
    search_bar.send_keys(Keys.ENTER)
    result = None
    while result is None:
        try:
            pagemax = int(driver.find_element("class name", "cst-pagination__label").text[5:])
            result = 1
        except:
            pass
    i = 1
    url = []
    while i <= 3: #Il faut le mettre à pagemax !!
        offers = driver.find_elements("class name", "cst-card")
        for offer in offers:
            link = offer.find_element("class name", "cst-card__link").get_attribute('href')
            url.append(link)
        next_page = driver.find_element("xpath", "//button[text()='Next']")
        next_page.click()
        sleep(1)
        i += 1
    links = pd.DataFrame(url, columns=['url'])
    return links

On fait ici la fonction qui va prendre les infos que l'on veut dans un article : 

In [329]:
def getNews(article_url : str) : 
    try : 
        driver.get(article_url)
    except : 
        return None 
    article = dict()
    driver.get(article_url)
    article = dict()
    article["headline"] = driver.find_element("tag name", 'h1').text
    text = ''
    for element in  driver.find_elements("tag name", 'p'):
        text += element.text
    article["fulltext"] = text
    article["url"]= article_url
    return article

In [215]:
getNews('https://www.telegraph.co.uk/news/2021/10/23/facebook-wants-new-rules-internet-arent-waiting/')

{'author': 'MONIKA BICKERT',
 'headline': 'Facebook wants new rules for the internet, but we aren’t waiting',
 'fulltext': "The internet has transformed the world over the last two decades, but it has also introduced new challenges, and legislation has not kept up. This week Parliament will discuss new rules for harmful content online with both tech companies and some of their staunchest critics. While there will no doubt be differing views, we should all agree on one thing: the tech industry needs regulation.At Facebook, we’ve advocated for democratic governments to set new rules for the internet on areas like harmful content, privacy, data and elections, because we believe that businesses like ours should not be making these decisions on our own. The UK is one of the countries leading the way with wide-ranging proposals on everything from hate speech to child safety and, while we won’t agree with all the details, we’re pleased the Online Safety Bill is moving forward.I spent more tha

In [345]:
def articles_from_theme_TE(theme : str):
    df_links = getLinks(theme) # Le Telegraph ne fait pas de différence entre majuscules ou pas dans l'url
    articles = []
    for ind in df_links.index :
        news = getNews(df_links.iloc[ind,0])
        articles.append(news)
    df_news = pd.DataFrame(articles)
    return df_news

In [346]:
articles_from_theme_TE('facebook')

,headline,fulltext,url
0,Facebook removes sanitary towel ad for saying ...,Bodyform said the video was part of an anti-st...,https://www.telegraph.co.uk/news/2023/06/08/fa...
1,Facebook is all power with no transparency – a...,Silicon Valley has no incentive to protect fre...,https://www.telegraph.co.uk/news/2023/06/02/fa...
2,The brutal workout that made Mark Zuckerberg s...,"Facebook boss performs hundreds of squats, pus...",https://www.telegraph.co.uk/world-news/2023/05...
3,Teacher claims he was sacked for defending his...,The alleged victim of a gang of youngsters pos...,https://www.telegraph.co.uk/news/2023/05/30/te...
4,EU accused of breaking up the internet after F...,Nick Clegg warns ruling sets a ‘dangerous prec...,https://www.telegraph.co.uk/business/2023/05/2...
5,Facebook to follow Twitter with monthly blue t...,UK users can secure a ‘verified’ blue tick if ...,https://www.telegraph.co.uk/business/2023/05/1...
6,Mark Zuckerberg wipes the floor with rivals in...,"The 38-year-old Facebook chief, described by h...",https://www.telegraph.co.uk/world-news/2023/05...
7,Mirror publisher blames Mark Zuckerberg for sa...,Reach's ad revenues down by almost a fifth aft...,https://www.telegraph.co.uk/business/2023/05/0...
8,Boost for Mark Zuckerberg as Facebook-owner Me...,"However, losses at the tech giant's virtual re...",https://www.telegraph.co.uk/business/2023/04/2...
9,Tom Tugendhat: Facebook and Instagram’s new pr...,The social media giants are adopting end-to-en...,https://www.telegraph.co.uk/politics/2023/04/1...


# Fonctions

In [347]:
def getLinks(theme : str) :
    driver.get("https://www.telegraph.co.uk/search/")
    search_bar = driver.find_element("name", "fsearch")
    search_bar.send_keys(theme)
    search_bar.send_keys(Keys.ENTER)
    result = None
    while result is None:
        try:
            pagemax = int(driver.find_element("class name", "cst-pagination__label").text[5:])
            result = 1
        except:
            pass
    i = 1
    url = []
    while i <= 3: #Il faut le mettre à pagemax !!
        offers = driver.find_elements("class name", "cst-card")
        for offer in offers:
            link = offer.find_element("class name", "cst-card__link").get_attribute('href')
            url.append(link)
        next_page = driver.find_element("xpath", "//button[text()='Next']")
        next_page.click()
        sleep(1)
        i += 1
    links = pd.DataFrame(url, columns=['url'])
    return links

In [348]:
def getNews(article_url : str) : 
    try : 
        driver.get(article_url)
    except : 
        return None 
    article = dict()
    driver.get(article_url)
    article = dict()
    article["headline"] = driver.find_element("tag name", 'h1').text
    text = ''
    for element in  driver.find_elements("tag name", 'p'):
        text += element.text
    article["fulltext"] = text
    article["url"]= article_url
    return article

In [349]:
def articles_from_theme_TE(theme : str):
    df_links = getLinks(theme) # Le Telegraph ne fait pas de différence entre majuscules ou pas dans l'url
    articles = []
    for ind in df_links.index :
        news = getNews(df_links.iloc[ind,0])
        articles.append(news)
    df_news = pd.DataFrame(articles)
    return df_news

# DataFrame Facebook

In [350]:
df_news_fb = articles_from_theme_TE('facebook')

In [351]:
df_fb_TE = df_news_fb[df_news_fb['url'].str.contains('meta|facebook|messenger|instagram|oculus|whatsApp|zuckerberg|olivan|clegg|social-media|gafa|gafam|big-tech|consumer-privacy|tech')]

In [352]:
df_fb_TE

,headline,fulltext,url
0,Facebook removes sanitary towel ad for saying ...,Bodyform said the video was part of an anti-st...,https://www.telegraph.co.uk/news/2023/06/08/fa...
1,Facebook is all power with no transparency – a...,Silicon Valley has no incentive to protect fre...,https://www.telegraph.co.uk/news/2023/06/02/fa...
2,The brutal workout that made Mark Zuckerberg s...,"Facebook boss performs hundreds of squats, pus...",https://www.telegraph.co.uk/world-news/2023/05...
3,Teacher claims he was sacked for defending his...,The alleged victim of a gang of youngsters pos...,https://www.telegraph.co.uk/news/2023/05/30/te...
4,EU accused of breaking up the internet after F...,Nick Clegg warns ruling sets a ‘dangerous prec...,https://www.telegraph.co.uk/business/2023/05/2...
5,Facebook to follow Twitter with monthly blue t...,UK users can secure a ‘verified’ blue tick if ...,https://www.telegraph.co.uk/business/2023/05/1...
6,Mark Zuckerberg wipes the floor with rivals in...,"The 38-year-old Facebook chief, described by h...",https://www.telegraph.co.uk/world-news/2023/05...
7,Mirror publisher blames Mark Zuckerberg for sa...,Reach's ad revenues down by almost a fifth aft...,https://www.telegraph.co.uk/business/2023/05/0...
8,Boost for Mark Zuckerberg as Facebook-owner Me...,"However, losses at the tech giant's virtual re...",https://www.telegraph.co.uk/business/2023/04/2...
9,Tom Tugendhat: Facebook and Instagram’s new pr...,The social media giants are adopting end-to-en...,https://www.telegraph.co.uk/politics/2023/04/1...


# DataFrame Amazon 

In [353]:
df_news_am = articles_from_theme_TE('amazon')

In [354]:
df_am_TE = df_news_am[df_news_am['url'].str.contains('amazon|prime|whole-foods|zappos|pillpack|twitch|audible|goodreads|imdb|bezos|jassy|olsavsky|alexander|gafa|gafam|big-tech|consumer-privacy|tech')]

In [355]:
yy=df_am_TE[df_am_TE['fulltext'].str.contains('forest|tribe|jungle|swallows-and-amazon|amazons')]
df_am_TE.drop(yy.index,inplace=True)

# DataFrame Apple

In [360]:
df_news_ap = articles_from_theme_TE('apple')

In [361]:
df_ap_TE = df_news_ap[df_news_ap['url'].str.contains('apple|iphone|iPad|mac|watch|macbook|ios|airpods|app-store|itunes|icloud|beats|siri|shazam|cook|jobs|gafa|gafam|big-tech|consumer-privacy|tech')]

In [362]:
yy=df_ap_TE[df_ap_TE['fulltext'].str.contains('apples|pie|pies|recipe|tart|crumble|bake|vinegar')]
df_ap_TE.drop(yy.index,inplace=True)

# DataFrame Google

In [364]:
df_news_go = articles_from_theme_TE('google')

In [365]:
df_go_TE = df_news_go[df_news_go['url'].str.contains('google|android|chrome|gmail|maps|playstore|pixel|waze|youTube|alphabet|mandiant|fitbit|looker|nest|doubleclick|page|brin|pichai|kurian|gafa|gafam|big-tech|consumer-privacy|tech')]

# DataFrame Microsoft

In [366]:
df_news_mi = articles_from_theme_TE('microsoft')

In [367]:
df_mi_TE = df_news_mi[df_news_mi['url'].str.contains('microsoft|windows|cortana|excel|explorer|office|edge|teams|outlook|powerpoint|skype|surface|word|xbox|linkedIn|github|mojang|gates|nadella|gafa|gafam|big-tech|consumer-privacy|tech')]

# DataFrame final 

In [ ]:
from google.colab import files
df_final = pd.concat([df_fb_TE, df_am_TE, df_ap_TE, df_go_TE, df_mi_TE]).reset_index()
df_final.drop_duplicates(subset ="url", keep = 'first', inplace=True)
df_final.to_csv('articlesTE.csv') 
#files.download('articlesGUA.csv')

In [ ]:
while True:pass